In [1]:
# Big input
input_file = "/home/bert/Downloads/05.txt"
# input_file = "input.txt"
with open(input_file, "r") as f:
    line1, *rest = f.read().split("\n\n")
seeds = [int(s) for s in line1.split()[1:]]
ranges = [(seeds[s], seeds[s] + seeds[s+1]) for s in range(0, len(seeds), 2)]

maps = [
    sorted(
        ((s := [int(i) for i in l.split()])[1], s[2], s[0])
        for l in map.split("\n")[1:]
    )
    for map in rest
]

In [2]:
from bisect import bisect_left, bisect_right

def precompute_start_stop(m):
    start = []
    stop = []
    for i, k, _ in m:
        start.append(i)
        stop.append(i + k)
    return start, sorted(stop)

def part1(seeds, maps=maps):
    for m in maps:
        out = []
        start, stop = precompute_start_stop(m)
        for seed in seeds:
            for n in range(bisect_right(stop, seed), bisect_right(start, seed)):
                source, length, dest = m[n]
                if source <= seed < source + length:
                    out.append(dest + seed - source)
                    break
            else:
                out.append(seed)
        seeds = out
    return seeds

print(min(part1(seeds)))
%timeit min(part1(seeds))

14876664
37.2 ms ± 4.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
import collections
from heapq import heappush, heappop

def part2(ranges, maps=maps):
    v = collections.deque(ranges)
    w = []
    for m in maps:
        start, stop = precompute_start_stop(m)
        while v:
            a, b = v.popleft()
            for n in range(bisect_right(stop, a), bisect_left(start, b)):
                i, k, j = m[n]
                off = j - i
                l = i + k                
                if i <= a < l:
                    if l < b:
                        v.append((l, b))
                        heappush(w, (a + off, l + off))
                    else:
                        heappush(w, (a + off, b + off))
                    break
                elif i < b <= l:
                    v.append((a, i))
                    heappush(w, (i + off, b + off))
                    break
                elif a < i and l < b:
                    v.append((a, i))
                    v.append((l, b))
                    heappush(w, (i + off, l + off))
                    break
            else:
                heappush(w, (a, b))
        last_a, last_b = heappop(w)
        while w:
            a, b = heappop(w)
            if a > last_b:
                v.append((last_a, last_b))
                last_a = a
                last_b = b
            else:
                last_b = b
        v.append((last_a, last_b))
    return v

print(min(part2(ranges))[0])
%timeit min(part2(ranges))

441394
99.9 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
